In [ ]:
!pip install xlrd

In [ ]:
!pip install pandas

In [ ]:
!pip install xgboost

In [ ]:
!pip install -U tensorflow

In [ ]:
!pip install scikit-learn

In [1]:
import pandas as pd
import numpy as np

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

# Load Reynolds

## load daily soil temperature

In [2]:
import os
fdir = "data/Reynolds/soiltemperature"

def loadReynolds(fname):
    df = pd.read_csv(fname, delim_whitespace=True,comment="#", encoding="ISO-8859-1",on_bad_lines="warn",header=None)
    df = df[df[3]==17]
    df_time = pd.to_datetime(df[0].astype(str) +  ' ' + df[1].astype(str) + ' ' + df[2].astype(str),format="%m %d %Y")
    
    df2 = df[[5,6,7,8,9,10,11,12]].replace('.', np.nan).astype(float).ffill(axis=0)

    df2[1] = fname.replace(fdir,"").replace("/hourly","").replace("soiltemperature.txt","")
    
    return pd.concat([df_time,df2], axis=1) 

soil_temp = []
for file in os.listdir(fdir):
    if file.endswith(".txt") and file.find("hourly") > -1:
        print(file)
        path = os.path.join(fdir, file)
        soil_temp += [loadReynolds(path)]

reynolds_soil_temp = pd.concat(soil_temp, axis=0)

hourly127x07soiltemperature.txt
hourly098x97soiltemperature.txt
hourly076x59soiltemperature.txt
hourly057x96soiltemperature.txt
hourly176x14soiltemperature.txt


In [3]:
fdir2 = "data/Reynolds"

def loadReynoldsCL(fname):
    df = pd.read_csv(fname, delim_whitespace=True,comment="#", encoding="ISO-8859-1",on_bad_lines="warn",header=None)
    df = df[(df[2] > 1984)|(df[2] == 1984) & (df[0] == 12) &(df[1] > 4)]
    df_time = pd.to_datetime(df[0].astype(str) +  ' ' + df[1].astype(str) + ' ' + df[2].astype(str),format="%m %d %Y")
    
    df2 = df[[3,4]].replace('.', np.nan).astype(float).ffill(axis=0)

    df2[1] = fname.replace(fdir2,"").replace("/daily","").replace("climate.txt","")
    
    return pd.concat([df_time,df2], axis=1) 

climate = []
for file in os.listdir(fdir2):
    if file.endswith(".txt") and file.find("daily") > -1:
        print(file)
        path = os.path.join(fdir2, file)
        climate += [loadReynoldsCL(path)]

reynolds_climate = pd.concat(climate, axis=0)

daily176x14climate.txt
daily127x07climate.txt
daily076x59climate.txt


In [4]:
reynolds_soil_temp = reynolds_soil_temp.rename(columns={0:'DATE',1:"LOC",5:"y_1",6:"y_2",7:"y_3",8:"y_4",9:"y_5",10:"y_6",11:"y_7",12:"y_8",3: 20, 4: 21}) 
reynolds_climate = reynolds_climate.rename(columns={0:'DATE',1:"LOC"})
climate = reynolds_climate.set_index(['DATE','LOC']).rename(columns={3: "C1", 4: "C2"}) 
soil = reynolds_soil_temp.set_index(['DATE','LOC'])

In [5]:
all_reynolds_data = pd.merge(soil,climate,left_index=True, right_index=True)

In [6]:
all_reynolds_data.to_csv("all_reynolds_data.csv",sep=";")

In [7]:
all_reynolds_data

,,y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,C1,C2
DATE,LOC,,,,,,,,,,
1984-12-05,127x07,NaN,0.4,NaN,2.0,NaN,NaN,NaN,NaN,-9.8,-2.4
1984-12-06,127x07,NaN,0.3,NaN,1.9,NaN,NaN,NaN,NaN,-9.4,-1.0
1984-12-07,127x07,NaN,0.2,NaN,1.8,NaN,NaN,NaN,NaN,-5.2,3.6
1984-12-08,127x07,NaN,0.1,NaN,1.7,NaN,NaN,NaN,NaN,-3.6,4.0
1984-12-09,127x07,NaN,0.2,NaN,1.6,NaN,NaN,NaN,NaN,-0.1,5.1
...,...,...,...,...,...,...,...,...,...,...,...
1996-09-26,176x14,15.4,12.5,9.5,9.6,10.2,10.4,10.8,11.0,-1.1,9.6
1996-09-27,176x14,18.4,14.8,10.3,10.2,10.2,10.4,10.7,10.9,1.9,17.0
1996-09-28,176x14,19.9,16.6,12.0,11.7,11.0,11.0,10.9,10.9,10.1,21.2


# UK

In [8]:
uk_data = "data/UK/catalogue.ceh.ac.uk/datastore/eidchub/1/"
climate_soil = []

def loadUK(fname):
    df = pd.read_csv(fname, sep=",",comment="#", on_bad_lines="warn")

    df1 = df.iloc[:, :2]
    df2 = df.iloc[:, 2:]    
    df2 = df2.astype(float).replace(-9999.0, np.nan).ffill(axis=0)    
    
    return pd.concat([df1,df2], axis=1)
    

for file in os.listdir(uk_data):
    if file.endswith(".csv") and file.find("daily") > -1 and file.find("flags") == -1 and file.find("metadata") == -1:
        print(file)
        path = os.path.join(uk_data, file)
        climate_soil += [loadUK(path)]

climate_soil =  pd.concat(climate_soil, axis=0)
climate_soil = climate_soil.rename(columns={"DATE_TIME":"DATE","SITE_ID":"LOC","TDT1_TSOIL":"y_1","TDT2_TSOIL":"y_2","TDT3_TSOIL":"y_3","TDT4_TSOIL":"y_4","TDT5_TSOIL":"y_5","TDT6_TSOIL":"y_6","TDT7_TSOIL":"y_7","TDT8_TSOIL":"y_8","TDT9_TSOIL":"y_9","TDT10_TSOIL":"y_10"}) 
climate_soil['DATE']= pd.to_datetime(climate_soil['DATE'])
climate_soil.set_index(["DATE","LOC"])

cosmos-uk_alic1_hydrosoil_daily_2015-2022.csv
cosmos-uk_tadhm_hydrosoil_daily_2014-2022.csv
cosmos-uk_crich_hydrosoil_daily_2014-2022.csv
cosmos-uk_lizrd_hydrosoil_daily_2014-2022.csv
cosmos-uk_morly_hydrosoil_daily_2014-2022.csv
cosmos-uk_redhl_hydrosoil_daily_2016-2022.csv
cosmos-uk_sourh_hydrosoil_daily_2014-2022.csv
cosmos-uk_plynl_hydrosoil_daily_2014-2022.csv
cosmos-uk_wrttl_hydrosoil_daily_2017-2022.csv
cosmos-uk_sydlg_hydrosoil_daily_2018-2022.csv
cosmos-uk_glens_hydrosoil_daily_2014-2022.csv
cosmos-uk_holln_hydrosoil_daily_2014-2022.csv
cosmos-uk_hartw_hydrosoil_daily_2014-2022.csv
cosmos-uk_hillb_hydrosoil_daily_2016-2022.csv
cosmos-uk_bickl_hydrosoil_daily_2015-2022.csv
cosmos-uk_cardt_hydrosoil_daily_2015-2022.csv
cosmos-uk_moorh_hydrosoil_daily_2014-2022.csv
cosmos-uk_eustn_hydrosoil_daily_2016-2022.csv
cosmos-uk_eastb_hydrosoil_daily_2014-2022.csv
cosmos-uk_riseh_hydrosoil_daily_2016-2022.csv
cosmos-uk_hybry_hydrosoil_daily_2017-2022.csv
cosmos-uk_rothd_hydrosoil_daily_20

,,LWIN,LWOUT,SWIN,SWOUT,RN,PRECIP,PRECIP_TIPPING,PRECIP_RAINE,PA,TA,...,STP_TSOIL50,COSMOS_VWC,CTS_MOD_CORR,D86_75M,SNOW,SNOW_DEPTH,SWE,ALBEDO,PE,GCC
DATE,LOC,,,,,,,,,,,,,,,,,,,,,
2015-03-06,ALIC1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-03-07,ALIC1,23.7,30.5,13.3,1.9,4.6,0.0,NaN,NaN,1012.2,8.6,...,5.6,40.8,1129.96052,22.50153,0.0,NaN,NaN,0.126,1.5,NaN
2015-03-08,ALIC1,27.1,30.5,4.8,0.7,0.7,0.0,NaN,NaN,1010.3,8.4,...,5.7,42.9,1120.16381,22.02295,0.0,NaN,NaN,0.116,0.4,NaN
2015-03-09,ALIC1,28.3,29.7,2.4,0.4,0.7,0.1,NaN,NaN,1011.8,6.2,...,5.9,38.8,1140.46891,22.99462,0.0,NaN,NaN,0.111,0.2,NaN
2015-03-10,ALIC1,23.5,29.8,12.4,1.8,4.3,0.0,NaN,NaN,1016.6,7.2,...,5.9,45.9,1106.90799,21.40022,0.0,NaN,NaN,0.121,1.4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,STIPS,28.4,29.0,0.4,0.1,-0.3,2.9,NaN,NaN,962.5,4.7,...,6.3,52.5,1317.61816,32.53471,0.0,NaN,5.56238,0.139,0.1,0.335
2022-12-28,STIPS,29.1,29.8,0.6,0.1,-0.2,6.4,NaN,NaN,945.7,6.4,...,6.2,52.5,1317.61816,32.53471,0.0,NaN,5.56238,0.138,0.3,0.334
2022-12-29,STIPS,26.3,28.7,1.7,0.4,-1.1,2.5,NaN,NaN,946.7,4.1,...,6.3,52.5,1317.61816,32.53471,0.0,NaN,5.56238,0.189,0.4,0.334


In [9]:
climate_soil.rename(columns={"DATE_TIME":"DATE","SITE_ID":"LOC","TDT1_TSOIL":"y_1","TDT2_TSOIL":"y_2","TDT3_TSOIL":"y_3","TDT4_TSOIL":"y_4","TDT5_TSOIL":"y_5","TDT6_TSOIL":"y_6","TDT7_TSOIL":"y_7","TDT8_TSOIL":"y_8","TDT9_TSOIL":"y_9","TDT10_TSOIL":"y_10"}) 
climate_soil = climate_soil.set_index(["DATE","LOC"])


In [10]:
climate_soil.to_csv("uk_soil.csv",sep=";")

In [11]:
climate_soil

,,LWIN,LWOUT,SWIN,SWOUT,RN,PRECIP,PRECIP_TIPPING,PRECIP_RAINE,PA,TA,...,STP_TSOIL50,COSMOS_VWC,CTS_MOD_CORR,D86_75M,SNOW,SNOW_DEPTH,SWE,ALBEDO,PE,GCC
DATE,LOC,,,,,,,,,,,,,,,,,,,,,
2015-03-06,ALIC1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-03-07,ALIC1,23.7,30.5,13.3,1.9,4.6,0.0,NaN,NaN,1012.2,8.6,...,5.6,40.8,1129.96052,22.50153,0.0,NaN,NaN,0.126,1.5,NaN
2015-03-08,ALIC1,27.1,30.5,4.8,0.7,0.7,0.0,NaN,NaN,1010.3,8.4,...,5.7,42.9,1120.16381,22.02295,0.0,NaN,NaN,0.116,0.4,NaN
2015-03-09,ALIC1,28.3,29.7,2.4,0.4,0.7,0.1,NaN,NaN,1011.8,6.2,...,5.9,38.8,1140.46891,22.99462,0.0,NaN,NaN,0.111,0.2,NaN
2015-03-10,ALIC1,23.5,29.8,12.4,1.8,4.3,0.0,NaN,NaN,1016.6,7.2,...,5.9,45.9,1106.90799,21.40022,0.0,NaN,NaN,0.121,1.4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,STIPS,28.4,29.0,0.4,0.1,-0.3,2.9,NaN,NaN,962.5,4.7,...,6.3,52.5,1317.61816,32.53471,0.0,NaN,5.56238,0.139,0.1,0.335
2022-12-28,STIPS,29.1,29.8,0.6,0.1,-0.2,6.4,NaN,NaN,945.7,6.4,...,6.2,52.5,1317.61816,32.53471,0.0,NaN,5.56238,0.138,0.3,0.334
2022-12-29,STIPS,26.3,28.7,1.7,0.4,-1.1,2.5,NaN,NaN,946.7,4.1,...,6.3,52.5,1317.61816,32.53471,0.0,NaN,5.56238,0.189,0.4,0.334


In [12]:
climate_soil = climate_soil.drop(["SNOW_DEPTH", "TDT1_VWC","TDT2_VWC","TDT3_VWC","TDT4_VWC","TDT5_VWC","TDT6_VWC","TDT7_VWC","TDT8_VWC","TDT9_VWC","TDT10_VWC","PRECIP_TIPPING","PRECIP_RAINE"],axis=1)

# Preprocessing of the Datasets

### Split onto 28 day long fragments. Remove fragments, which do not containt more than 30% of target values. 

### We do not consider temperatures for other soil layers as fatures deliberatly. 

In [13]:
from datetime import date, timedelta
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

PERIOD = 28
TRHX = 0.3

def daterange(start_date, end_date, step):
    for n in range(0,int((end_date - start_date).days), step):
        yield start_date + timedelta(n)

all_data = []
for df in [all_reynolds_data, climate_soil]:
    locations = df.index.get_level_values(1)
    
    rdatasX = {}
    rdatasY = {}
    for l in set(locations):
        ld = df.query("LOC == '" + l + "'").reset_index().set_index("DATE").drop("LOC",axis=1)
        all_start = ld.index.min()
        all_end = ld.index.max()
    
        for start in daterange(all_start, all_end, PERIOD):
            end = start + timedelta(PERIOD)
            period_data = ld.loc[start:end]
            
            y_columns = set([c for c in df.columns if c.find("y_") > -1])
            not_y_columns = [c for c in df.columns if c.find("y_") == -1]
            for y_counter in y_columns:
                if y_counter not in rdatasX: 
                    rdatasX[y_counter] = []
                    rdatasY[y_counter] = []
                    
                y_ = period_data[y_counter].to_numpy().astype(float)
                y = y_[1:]
    
                nans = np.count_nonzero(np.isnan(y))
    
                if float(nans) / y.shape[0] < TRHX and y.shape[0] == PERIOD:
                    X = period_data[not_y_columns].to_numpy()[:-1].astype(float)
                    X = np.hstack([X,y_[:-1].reshape(-1,1)]) # add current y as a feature
                    old_dim = X.shape[1]
                    X = imp.fit_transform(X)
                    if X.shape[1] == old_dim:
                        nans = np.count_nonzero(np.isnan(X))
                        if nans == 0:
                            y = imp.fit_transform(y.reshape(-1,1)).reshape(y.shape)
                            rdatasX[y_counter].append(X)
                            rdatasY[y_counter].append(y)
                    
    all_data.append({k:[np.asarray(rdatasX[k]), np.asarray(rdatasY[k])] for k in rdatasX})

In [14]:
all_data[0].keys()

dict_keys(['y_2', 'y_7', 'y_8', 'y_5', 'y_6', 'y_1', 'y_4', 'y_3'])

In [15]:
from sklearn.preprocessing import normalize

for k in all_data[0]:
    all_data[0][k][0] = normalize(all_data[0][k][0].reshape(-1,all_data[0][k][0].shape[2]),axis=0).reshape(all_data[0][k][0].shape)
    
for k in all_data[1]:
    all_data[1][k][0] = normalize(all_data[1][k][0].reshape(-1,all_data[1][k][0].shape[2]),axis=0).reshape(all_data[1][k][0].shape)

In [16]:
from sklearn.model_selection import train_test_split
dict_data = {}
dict_data["Reynolds"] = {}
dict_data["UK"] = {}

for k in all_data[0]:
    x01,x02,y01,y02 = train_test_split(all_data[0][k][0], all_data[0][k][1], test_size=0.3,random_state=42)
    dict_data["Reynolds"][k] = {"train":{"X":x01,"y":y01},"test":{"X":x02,"y":y02}}
    
for k in all_data[1]:
    x11,x12,y11,y12 = train_test_split(all_data[1][k][0], all_data[1][k][1],test_size=0.3,random_state=42)
    dict_data["UK"][k] = {"train":{"X":x11,"y":y11},"test":{"X":x12,"y":y12}} 

all_data = dict_data


# Find hyperparameters

In [ ]:
!pip install xgboost

In [ ]:
!pip install protobuf==4.21.12

In [ ]:
!pip install -U tkan

In [ ]:
!pip install optuna

In [ ]:
!pip install scikit-learn

## Neural Networks

In [ ]:
import tensorflow as tf
import optuna
from tkan import TKAN, BSplineActivation
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from tensorflow.keras import regularizers
from sklearn.model_selection import KFold

nn_data = []


def make_modelLSTM(input_shape, hidden_size, dropout):
    input_layer = tf.keras.layers.Input(input_shape)
    #dim = tf.zeros([batch_size,hidden_size])  
    output_layer = tf.keras.layers.LSTM(hidden_size, return_sequences=True,dropout=dropout)(input_layer)#, initial_state=[dim, dim])
    output_layer2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation='linear'))(output_layer)    
    return tf.keras.models.Model(inputs=input_layer, outputs=output_layer2)
 
def make_GRU(input_shape, hidden_size, dropout):
    input_layer = tf.keras.layers.Input(input_shape)
    output_layer = tf.keras.layers.GRU(hidden_size, return_sequences=True,dropout=dropout)(input_layer)#, initial_state=[dim, dim])
    output_layer2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation='linear'))(output_layer)    
    return tf.keras.models.Model(inputs=input_layer, outputs=output_layer2)

def make_modelTKAN(input_shape, hidden_size, dropout):
    model = tf.keras.Sequential([
          tf.keras.layers.InputLayer(input_shape=input_shape),
          TKAN(hidden_size, tkan_activations=[BSplineActivation(3)], return_sequences=True, use_bias=True, kernel_regularizer=regularizers.L2(dropout),
    bias_regularizer=regularizers.L2(dropout), recurrent_regularizer=regularizers.L2(dropout)),
          tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation='linear')),
    ])
    return model

#models = {"LSTM":make_modelLSTM, "GRU": make_GRU, "TKAN": make_modelTKAN}
models = {"TKAN": make_modelTKAN}

for model_name in models:
    make_model = models[model_name]
    #for ds_name in all_data:
    for ds_name in ["UK"]:    
        #for depth in all_data[ds_name]:
        for depth in ['y_9']:    
            dat = all_data[ds_name][depth]
            x_train = dat["train"]["X"]
            x_test = dat["test"]["X"]
            Y_train = dat["train"]["y"]
            Y_test = dat["test"]["y"]
            batch_size = 4       
    
            def objective(trial):
                lr = trial.suggest_float('lr', 0.0001, 0.1)
                hidden_size = trial.suggest_int('hs', 2, 8)
                en = trial.suggest_int('en', 10, 500)

                if model_name == "TKAN":
                    do = trial.suggest_float('dropout', 1e-5, 1e-2)
                else:    
                    do = trial.suggest_float('dropout', 0.05, 0.2)
                    
                kf = KFold(n_splits=3)
                scores = []
                for _, (train_index, test_index) in enumerate(kf.split(x_train)):
                    model = make_model(input_shape=x_train.shape[1:],hidden_size=hidden_size, dropout = do)
                # 
                    opt = tf.keras.optimizers.Adam(learning_rate=lr)
                
                    model.compile(
                         optimizer=opt,
                         loss="mean_squared_error",
                         metrics=["mean_squared_error"],
                    )
                    
                    history = model.fit(
                         x_train[train_index],
                         Y_train[train_index],
                         batch_size=batch_size,
                         epochs=en,
                         verbose=0,
                    )
                    try:
                        y_pred = model.predict(x_train[test_index]) #, batch_size=batch_size)
                        scores.append(mean_squared_error(Y_train[test_index].flatten(),y_pred.flatten()))
                    except:
                        scores.append(500)
                return np.asarray(scores).mean() 
                
            study = optuna.create_study(direction='minimize')
            study.optimize(objective, n_trials=20)    
            
            lr = study.best_trial.params["lr"]     
            hs = study.best_trial.params["hs"]     
            en = study.best_trial.params["en"]   
            do = study.best_trial.params["dropout"]   
    
            model = make_model(input_shape=x_train.shape[1:],hidden_size=hs, dropout = do)
        # 
            opt = tf.keras.optimizers.Adam(learning_rate=lr)
        
            model.compile(
                 optimizer=opt,
                 loss="mean_squared_error",
                 metrics=["mean_squared_error"],
            )
            
            history = model.fit(
                 x_train,
                 Y_train,
                 batch_size=batch_size,
                 epochs=en,
                 verbose=0,
            )        
            try:
                y_pred = model.predict(x_test) #, batch_size=batch_size)
                mse_score = mean_squared_error(Y_test.flatten(),y_pred.flatten())
                mae_score = mean_absolute_error(Y_test.flatten(),y_pred.flatten())
                print(model_name,ds_name,depth,mse_score, mae_score, Y_test.min(),Y_test.max())     
                nn_data.append([model_name,ds_name,depth,mse_score, mae_score])
            except:
                pass



## Boosting

In [ ]:
!pip install deep-forest

In [ ]:
!pip install cython

In [ ]:
!pip install optuna

In [ ]:
all_data["Reynolds"].keys()

In [ ]:
import xgboost as xgb
from boosted_forest import CascadeBoostingRegressor
from deepforest import CascadeForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import optuna

from sklearn.model_selection import KFold
#xgb.set_config(verbosity=2)

def make_modelXGB(max_depth,layers,C):
    return xgb.XGBRegressor(max_depth = max_depth, n_estimators = layers)

def make_modelCascade(max_depth,layers,C):
    return CascadeForestRegressor(max_depth = max_depth, max_layers = layers, n_estimators=4)

def make_modelBoosted(max_depth,layers,C):
    return CascadeBoostingRegressor(C=C, n_layers=layers, n_estimators = 4, max_depth=max_depth, n_iter_no_change = 1, validation_fraction = 0.1, learning_rate = 0.9)


#models = {"XGB":make_modelXGB,"Cascade Forest":make_modelCascade, "Boosted Forest": make_modelBoosted}
models = {"Boosted Forest": make_modelBoosted}

bo_data = []    

for model_name in models:
    make_model = models[model_name]
    for ds_name in ["Reynolds"]:
        for depth in ['y_7', 'y_6', 'y_5']:
            dat = all_data[ds_name][depth]
            x_train = dat["train"]["X"].reshape(-1,dat["train"]["X"].shape[2])
            x_test = dat["test"]["X"].reshape(-1,dat["test"]["X"].shape[2])
            Y_train = dat["train"]["y"].flatten()
            Y_test = dat["test"]["y"].flatten()            

            def objective(trial):
                layers = trial.suggest_int('layers', 5, 15)
                max_depth = trial.suggest_int('max_depth', 1, 3)

                if model_name == "Boosted Forest":
                    C = trial.suggest_int('C', 1, 2000)
                else:
                    C = 0

                kf = KFold(n_splits=3)
                scores = []
                for _, (train_index, test_index) in enumerate(kf.split(x_train)):
                    model = make_model(max_depth,layers,C)
                    
                    model.fit(
                         x_train[train_index],
                         Y_train[train_index],
                    )
                    y_pred = model.predict(x_train[test_index]) #, batch_size=batch_size)
                    scores.append(mean_squared_error(Y_train[test_index].flatten(),y_pred.flatten()))
                return np.asarray(scores).mean() 
            
            study = optuna.create_study(direction='minimize')
            study.optimize(objective, n_trials=20)    
            
            layers = study.best_trial.params["layers"]  
            max_depth = study.best_trial.params["max_depth"]  

            if model_name == "Boosted Forest":
                C = study.best_trial.params["C"]  
            else:
                C = 0
            model = make_model(max_depth,layers,C)
            model.fit(
                 x_train,
                 Y_train,
            )        
            
            y_pred = model.predict(x_test) #, batch_size=batch_size)
            mse_score = mean_squared_error(Y_test.flatten(),y_pred.flatten())
            mae_score = mean_absolute_error(Y_test.flatten(),y_pred.flatten())
            print(model_name,ds_name,depth,mse_score, mae_score, Y_test.min(),Y_test.max())     
            bo_data.append([model_name,ds_name,depth,mse_score, mae_score])
    

[I 2024-06-07 10:41:19,496] A new study created in memory with name: no-name-2e54c22d-2a0c-4581-a6a9-f709245e50d8


In [ ]:
import numpy
numpy.version.version

In [ ]:
!pip install -U numpy==1.22.4

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
rf_data = [] 

for ds_name in all_data:
    for depth in all_data[ds_name]:
        best = 1000
        best_d = 2
        for n_estimators in [100]:
            for tdepth in [2,3,5,7,10,12]:
                kf = KFold(n_splits=3)
                scores = []
                for _, (train_index, test_index) in enumerate(kf.split(x_train)):
                    model = RandomForestRegressor(n_estimators = n_estimators, max_depth = tdepth)
                    dat = all_data[ds_name][depth]
                    model.fit(dat["train"]["X"].reshape(-1,dat["train"]["X"].shape[2])[train_index], dat["train"]["y"].flatten()[train_index])
                    y_pred = model.predict(dat["train"]["X"].reshape(-1,dat["train"]["X"].shape[2]))[test_index])
                    scores.append(mean_squared_error(y_pred, dat["train"]["y"].flatten())[test_index])
                    
                if np.asarray(scores).mean() < best:
                    best = np.asarray(scores).mean()
                    best_d = tdepth

        model = RandomForestRegressor(n_estimators = n_estimators, max_depth = best_d)
        dat = all_data[ds_name][depth]
        model.fit(dat["train"]["X"].reshape(-1,dat["train"]["X"].shape[2]), dat["train"]["y"].flatten())
        y_pred = model.predict(dat["test"]["X"].reshape(-1,dat["test"]["X"].shape[2]))
        mse_score = mean_squared_error(y_pred, dat["test"]["y"].flatten())
        mae_score = mean_absolute_error(y_pred, dat["test"]["y"].flatten())        
                
        print("Random Forest", ds_name,depth,mse_score, mae_score, dat["test"]["y"].min(),dat["test"]["y"].max())
        rf_data.append(["Random Forest", ds_name,depth,mse_score, mae_score])

# Trash

In [ ]:
from boosted_forest import CascadeBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

for ds_name in ["UK"]: #all_data:
    for depth in ['y_5', 'y_1', 'y_10', 'y_7', 'y_8', 'y_9', 'y_6']:#all_data[ds_name]:
        model = CascadeBoostingRegressor(C=1000, n_layers=10, verbose=2, n_estimators = 4, max_depth=1, n_iter_no_change = 1, validation_fraction = 0.1, learning_rate = 0.9)
        dat = all_data[ds_name][depth]
        model.fit(dat["train"]["X"].reshape(-1,dat["train"]["X"].shape[2]), dat["train"]["y"].flatten())
        y_pred = model.predict(dat["test"]["X"].reshape(-1,dat["test"]["X"].shape[2]))
        mse_score = mean_squared_error(y_pred, dat["test"]["y"].flatten())
        mae_score = mean_absolute_error(y_pred, dat["test"]["y"].flatten())
        print(ds_name,depth,mse_score, mae_score, dat["test"]["y"].min(),dat["test"]["y"].max())
        del model
   

In [ ]:
Reynolds y_3 1.196124186330187 0.7492197406658464 -8.6 31.5

In [ ]:
x_train.shape[1:]

In [ ]:
batch_size

In [ ]:
x_train.shape

In [ ]:
for ds in all_data:
    for k in all_data[ds]:
        print(all_data[ds][k]['train']['X'].shape,all_data[ds][k]['test']['X'].shape)

In [ ]:
195 / 9

In [ ]:
!pip install optuna

In [ ]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

for ds_name in all_data:
    for depth in all_data[ds_name]:
        for n_estimators in [10, 30, 100]:
            for tdepth in [2,3,5,7,10,12]:
                model = RandomForestRegressor(n_estimators = n_estimators, max_depth = tdepth)
                dat = all_data[ds_name][depth]
                model.fit(dat["train"]["X"].reshape(-1,dat["train"]["X"].shape[2]), dat["train"]["y"].flatten())
                y_pred = model.predict(dat["test"]["X"].reshape(-1,dat["test"]["X"].shape[2]))
                mse_score = mean_squared_error(y_pred, dat["test"]["y"].flatten())
                mae_score = mean_absolute_error(y_pred, dat["test"]["y"].flatten())
                
                print(n_estimators, tdepth, ds_name,depth,mse_score, mae_score, dat["test"]["y"].min(),dat["test"]["y"].max())
                del model
        break
    break   


In [ ]:
all_data["UK"].keys()

# TKAN

In [ ]:
from tkan import TKAN, BSplineActivation
import tensorflow as tf

from tensorflow.keras import regularizers


import optuna

epochs = 200


def make_model(input_shape, hidden_size):
    model = tf.keras.Sequential([
          tf.keras.layers.InputLayer(input_shape=input_shape),
          TKAN(hidden_size, tkan_activations=[BSplineActivation(3)], return_sequences=True, use_bias=True, kernel_regularizer=regularizers.L2(1e-4),
    bias_regularizer=regularizers.L2(1e-4), recurrent_regularizer=regularizers.L2(1e-4)),
          tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation='linear')),
    ])
    return model
# 

#def objective(trial):

for ds_name in ["UK"]:
    for depth in all_data[ds_name]:
        dat = all_data[ds_name][depth]
        x_train = dat["train"]["X"]
        x_test = dat["test"]["X"]
        Y_train = dat["train"]["y"]
        Y_test = dat["test"]["y"]
        batch_size = 4       
        model = make_model(input_shape=x_train.shape[1:],hidden_size=8)
        # 
        opt = tf.keras.optimizers.Adam(learning_rate=0.01)
        
        model.compile(
             optimizer=opt,
             loss="mean_squared_error",
             metrics=["mean_squared_error"],
        )
        
        history = model.fit(
             x_train,
             Y_train,
             batch_size=batch_size,
             epochs=epochs,
             verbose=0,
        )
        # 
        #batch_size = x_test.shape[0]
        y_pred = model.predict(x_test) #, batch_size=batch_size)

        try:
            mse_score = mean_squared_error(Y_test.flatten(),y_pred.flatten())
            mae_score = mean_absolute_error(Y_test.flatten(),y_pred.flatten())
            print(ds_name,depth,mse_score, mae_score, Y_test.min(),Y_test.max())     
        except:
            print(ds_name,depth," test error!")
        
        y_pred = model.predict(x_train) #, batch_size=batch_size)
        try:
            mse_score = mean_squared_error(Y_train.flatten(),y_pred.flatten())
            mae_score = mean_absolute_error(Y_train.flatten(),y_pred.flatten())
            print("train: ", ds_name,depth,mse_score, mae_score, Y_train.min(),Y_train.max()) 
        except:
            print(ds_name,depth," train error!")
